**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense,Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Conv3D

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Given as input the state of the agent, the function act returns the action to perform at this state. It can be either a random act with some probability epsilon, or otherwise a learned act from past experiences.
Epsilon is essentiel because it represents the trade-off between:
- the exploration state where the agent explore the actions to learn about them and possible rewards
- the exploitation state where the agent maximises its total reward

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #get the reward from the new state
        reward = self.board[self.x, self.y]
        #set the new state reward at 0
        self.board[self.x, self.y] = 0
        #check if the game is over 
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array **board** stores rewards associated to each cell (i.e cheese and poisonous cells locations).<br>
The array **position** stores all the possible position where a rat can go. It has -1 in the bondaries where the rat can't go, 1 in the position where the rat actually is, and 0 finally for the positions he can reach. 




## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over=False
        
        win=0
        lose=0
        
        while not game_over :
            # The agent performs an action. 
            # As we don't want to optimize the policy during a test phase, we put the train argument of the act method at False 
            action = agent.act(state, train=False)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e+1))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 7.5/6.0. Average score (1.5)
Win/lose count 11.5/10.0. Average score (1.5)
Win/lose count 9.0/14.0. Average score (-0.6666666666666666)
Win/lose count 12.5/17.0. Average score (-1.625)
Win/lose count 10.0/10.0. Average score (-1.3)
Win/lose count 7.0/14.0. Average score (-2.25)
Win/lose count 12.0/16.0. Average score (-2.5)
Win/lose count 11.5/26.0. Average score (-4.0)
Win/lose count 9.5/14.0. Average score (-4.055555555555555)
Win/lose count 8.5/10.0. Average score (-3.8)
Final score: -3.8


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}





Please check the file named 'Answer Question 5.pdf' in the same repository for those answers.

Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

This loss function comes from the previous equality. In fact, it allows to reduce the difference between the actual action-value function, and the optimal one. Thus, using this loss function, we try to verify the Bellman equation, by enforcing our Q-function to converge to the optimal one.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) >= self.max_memory :
            del self.memory[0]
        self.memory.append(m)
        pass

    def random_access(self):
        idx = np.random.randint(0, len(self.memory))
        return self.memory[idx]
        pass

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        action = np.argmax( self.model.predict(s.reshape((1, s.shape[0], s.shape[1], s.shape[2])))[0,:])
        
        
        return action
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            #  We get a random minibatch of transitions from the memory
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            input_states[i]= s_
            
            target_q[i] = self.model.predict(s_.reshape((1, s_.shape[0], s_.shape[1], s_.shape[2])))[0]
            
            if game_over_:
                ######## FILL IN
                target_q[i,a_]= r_
            else:
                ######## FILL IN
                target_q[i,a_]= r_ + self.discount * max(self.model.predict(n_s_.reshape((1, n_s_.shape[0], n_s_.shape[1], n_s_.shape[2])))[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(25, activation='relu'))
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), loss = "mse")
        self.model = model
        

In [12]:
#epochs_train= 100
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.05, epsilon = 0.15, memory_size=5000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0063 | Win/lose count 3.5/6.0 (-2.5)
Epoch 001/050 | Loss 0.0108 | Win/lose count 8.5/5.0 (3.5)
Epoch 002/050 | Loss 0.0179 | Win/lose count 3.0/6.0 (-3.0)
Epoch 003/050 | Loss 0.0191 | Win/lose count 6.0/5.0 (1.0)
Epoch 004/050 | Loss 0.0930 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/050 | Loss 0.0182 | Win/lose count 6.0/6.0 (0.0)
Epoch 006/050 | Loss 0.0215 | Win/lose count 4.0/4.0 (0.0)
Epoch 007/050 | Loss 0.0574 | Win/lose count 6.5/6.0 (0.5)
Epoch 008/050 | Loss 0.0252 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/050 | Loss 0.0107 | Win/lose count 4.0/3.0 (1.0)
Epoch 010/050 | Loss 0.0278 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/050 | Loss 0.0737 | Win/lose count 1.5/5.0 (-3.5)
Epoch 012/050 | Loss 0.0256 | Win/lose count 6.0/5.0 (1.0)
Epoch 013/050 | Loss 0.0059 | Win/lose count 6.0/5.0 (1.0)
Epoch 014/050 | Loss 0.0078 | Win/lose count 2.5/6.0 (-3.5)
Epoch 015/050 | Loss 0.0107 | Win/lose count 0/2.0 (-2.0)
Epoch 016/050 | Loss 0.0113 | Win/lose count 8.0/6.0 

After several tests, I came up with the following conclusions : <br>
- By increasing the espilon to 0.15 instead of 0.10, we have less chance going back and forth between 2 cells
- By increasing the memory size, we may increase the probability to achieve better score results
- 1 hidden layer achieves the same results than an architecture with 2 or 3 fully connected hidden layers. So we keep only 1 hidden layer.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(24,(2,2),input_shape=(5,5,self.n_state,),activation='relu', padding='same'))
        model.add(Conv2D(12,(2,2),activation='relu',padding='same'))
        model.add(Flatten())
        model.add(Dense(12, activation='relu'))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.05, epsilon = 0.15, memory_size=5000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/050 | Loss 0.0050 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/050 | Loss 0.0036 | Win/lose count 1.5/6.0 (-4.5)
Epoch 002/050 | Loss 0.0009 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/050 | Loss 0.0089 | Win/lose count 3.5/6.0 (-2.5)
Epoch 004/050 | Loss 0.0027 | Win/lose count 3.0/2.0 (1.0)
Epoch 005/050 | Loss 0.0010 | Win/lose count 1.0/2.0 (-1.0)
Epoch 006/050 | Loss 0.0104 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/050 | Loss 0.0058 | Win/lose count 2.0/2.0 (0.0)
Epoch 008/050 | Loss 0.0055 | Win/lose count 2.5/3.0 (-0.5)
Epoch 009/050 | Loss 0.0083 | Win/lose count 5.0/2.0 (3.0)
Epoch 010/050 | Loss 0.0109 | Win/lose count 5.5/3.0 (2.5)
Epoch 011/050 | Loss 0.0076 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/050 | Loss 0.0111 | Win/lose count 3.5/2.0 (1.5)
Epoch 013/050 | Loss 0.0056 | Win/lose count 4.5/1.0 (3.5)
Epoch 014/050 | Loss 0.0069 | Win/lose count 4.0/4.0 (0.0)
Epoch 015/050 | Loss 0.0135 | Win/lose count 3.0/6.0 (-3.0)
Epoch 016/050 | Loss 0.0016 | Win/lose count 1.5/3

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 2.5/0. Average score (1.1666666666666667)
Win/lose count 1.0/0. Average score (1.125)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0/0. Average score (0.8571428571428571)
Win/lose count 1.5/0. Average score (0.9375)
Win/lose count 1.0/0. Average score (0.9444444444444444)
Win/lose count 0/1.0. Average score (0.75)
Final score: 0.75
Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/4.0. Average score (-1.75)
Win/lose count 1.0/0. Average score (-0.8333333333333334)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 0.5/0. Average score (-0.3)
Win/lose count 0.5/1.0. Average score (-0.3333333333333333)
Win/lose count 0.5/0. Average score (-0.21428571428571427)
Win/lose count 0.5/0. Average score (-0.125)
Win/lose count 0/1.0. Average score (-0.2222222222222222)
Win/lose count 0.5/0. Average 

In [16]:
HTML(display_videos('cnn_test1.mp4'))

In [17]:
HTML(display_videos('fc_test2.mp4'))

**Observations :** <br>
One main issue we observe is that sometimes, the agents is kind of stuck between 2 cells, and keep going back and forth between those 2 cells. He doesn't explore the entire map. This particularly happens when the agent acts learns a rewardfull policy of going back and forth, and keep repeeting it.  <br>
By increasing the temperature, we observe that the scores increase. This is because the prabability to have rewards cells is higher. Also, there are sometimes less poisonous cells because of the definition of the map : if a cell is both poisonous and rewarding, we only keep it as rewarding.
<br>Moreover, the gap between the Fully-connected and CNN achitectures increase when the temperature increase. <br>Finally, the issue of the agent being stuck in some positions is less remarquable when the temperature is smaller.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        
        # We define a new state feature : malus_position
        # Each time the rat visits an already visited cells, he gets a malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size)) 
   
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #get the reward from the new state + If it's the train phase, gets a malus if the state has already been visited
        reward = 0
        if train :
            reward = -self.malus_position[self.x, self.y]
        reward += self.board[self.x, self.y]
        #set the new state reward at 0
        self.board[self.x, self.y] = 0
        #set the new state malus to 0.1
        self.malus_position[self.x, self.y] = 0.1
        #check if the game is over 
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        #Reset the maluses to 0
        self.malus_position = np.zeros((self.grid_size,self.grid_size)) 
        
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [19]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    dec_epsilon = agent.epsilon / epoch
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        #At each epoch, if epsilon >0.1, we decreache it
        if agent.epsilon > 0.1 :
            agent.set_epsilon(agent.epsilon - dec_epsilon )
        else: 
            agent.set_epsilon(0.1)
        
            
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_expert = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_expert, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0114 | Win/lose count 9.0/26.00000000000007 (-17.00000000000007)
Epoch 001/050 | Loss 0.0197 | Win/lose count 11.0/28.300000000000086 (-17.300000000000086)
Epoch 002/050 | Loss 0.0085 | Win/lose count 7.5/20.900000000000038 (-13.400000000000038)
Epoch 003/050 | Loss 0.0036 | Win/lose count 7.5/23.600000000000065 (-16.100000000000065)
Epoch 004/050 | Loss 0.0106 | Win/lose count 7.5/22.500000000000014 (-15.000000000000014)
Epoch 005/050 | Loss 0.0086 | Win/lose count 11.0/25.50000000000005 (-14.50000000000005)
Epoch 006/050 | Loss 0.0408 | Win/lose count 8.5/22.000000000000046 (-13.500000000000046)
Epoch 007/050 | Loss 0.0149 | Win/lose count 7.5/24.40000000000007 (-16.90000000000007)
Epoch 008/050 | Loss 0.0091 | Win/lose count 13.5/20.200000000000017 (-6.700000000000017)
Epoch 009/050 | Loss 0.0131 | Win/lose count 10.0/22.300000000000033 (-12.300000000000033)
Epoch 010/050 | Loss 0.0063 | Win/lose count 14.0/22.700000000000042 (-8.700000000000042)
Epoch 011/050

In [22]:
# Evaluation
test(agent_expert,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore1.mp4'))

Win/lose count 18.0/16.299999999999965. Average score (1.7000000000000348)
Win/lose count 10.0/15.299999999999962. Average score (-1.7999999999999634)
Win/lose count 13.0/12.89999999999997. Average score (-1.1666666666666323)
Win/lose count 25.0/10.29999999999998. Average score (2.800000000000031)
Win/lose count 8.0/17.399999999999977. Average score (0.3600000000000293)
Win/lose count 26.0/9.199999999999983. Average score (3.1000000000000276)
Win/lose count 19.5/13.399999999999968. Average score (3.5285714285714564)
Win/lose count 21.0/10.699999999999978. Average score (4.375000000000027)
Win/lose count 14.0/14.699999999999964. Average score (3.8111111111111393)
Win/lose count 22.0/9.599999999999982. Average score (4.6700000000000275)
Final score: 4.6700000000000275


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

We implemente a class Expert_mimicks(agent) that mimicks the behaviour of an expert agent

In [50]:
class Expert_mimicks(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Expert_mimicks, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])
    
    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        #Since our objective is to learn from the expert, we only need to store the current state, and the action taken from this state
        self.memory.remember([s_, a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            #  We get a random minibatch of transitions from the memory
            s_, a_  = self.memory.random_access()
            
            input_states[i]= s_
            ## As we want our agent to copy the expert, our target is the action of the expert. 
            #So we juste store a 5 in the target[i,a_], and assign 0 reward to other possible actions
            target_q[i,a_] = 5
            

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Expert_mimicks(Expert_mimicks):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Expert_mimicks, self).__init__(*args,**kwargs)
        ## We implement an architecture with 1 conv2D layer, et 1 fully connected layer
        # We use softmax as activation function for the last layer
        model = Sequential()
        model.add(Conv2D(25,(2,2),input_shape=(5,5,self.n_state,),activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(25))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [51]:
def train_explore_DQN_CNN_Expert_mimicks(agent,env,epoch,expert,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    dec_epsilon = agent.epsilon / epoch
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        #At each epoch, if epsilon >0.1, we decreache it
        if agent.epsilon > 0.1 :
            agent.set_epsilon(agent.epsilon - dec_epsilon )
        else: 
            agent.set_epsilon(0.1)
        
            
        while not game_over:
            # The experts performs an action
            action = expert.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [54]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_mimick = DQN_CNN_Expert_mimicks(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
train_explore_DQN_CNN_Expert_mimicks(agent_mimick, env, epochs_train,agent_expert, prefix='cnn_Expert_mimicks_train')
HTML(display_videos('cnn_Expert_mimicks_train10.mp4'))

Epoch 000/050 | Loss 1.8158 | Win/lose count 18.5/14.399999999999968 (4.100000000000032)
Epoch 001/050 | Loss 1.6119 | Win/lose count 17.0/13.099999999999971 (3.9000000000000288)
Epoch 002/050 | Loss 1.6704 | Win/lose count 27.0/15.499999999999982 (11.500000000000018)
Epoch 003/050 | Loss 2.3730 | Win/lose count 21.0/14.999999999999964 (6.0000000000000355)
Epoch 004/050 | Loss 3.4046 | Win/lose count 24.5/9.99999999999998 (14.50000000000002)
Epoch 005/050 | Loss 2.5615 | Win/lose count 19.0/16.099999999999977 (2.9000000000000234)
Epoch 006/050 | Loss 2.7558 | Win/lose count 20.0/13.399999999999972 (6.600000000000028)
Epoch 007/050 | Loss 2.8050 | Win/lose count 17.5/12.599999999999971 (4.900000000000029)
Epoch 008/050 | Loss 2.4287 | Win/lose count 22.0/12.799999999999974 (9.200000000000026)
Epoch 009/050 | Loss 2.2224 | Win/lose count 21.0/12.999999999999977 (8.000000000000023)
Epoch 010/050 | Loss 3.2027 | Win/lose count 20.0/12.79999999999998 (7.200000000000021)
Epoch 011/050 | Loss

In [57]:
# Evaluation
test(agent_mimick,env,epochs_test,prefix='cnn_Expert_mimicks_explore')
HTML(display_videos('cnn_Expert_mimicks_explore1.mp4'))

Win/lose count 19.0/15.899999999999967. Average score (3.1000000000000334)
Win/lose count 19.0/14.199999999999966. Average score (3.950000000000034)
Win/lose count 15.5/17.799999999999983. Average score (1.8666666666666956)
Win/lose count 5.0/18.39999999999999. Average score (-1.9499999999999762)
Win/lose count 2.0/19.60000000000001. Average score (-5.079999999999982)
Win/lose count 19.5/13.899999999999967. Average score (-3.29999999999998)
Win/lose count 7.0/17.799999999999983. Average score (-4.3714285714285515)
Win/lose count 9.0/16.89999999999997. Average score (-4.812499999999979)
Win/lose count 20.0/13.299999999999969. Average score (-3.533333333333311)
Win/lose count 14.5/17.099999999999973. Average score (-3.4399999999999773)
Final score: -3.4399999999999773


***